In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import heapq
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from enum import Enum

# Set style for better looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [2]:
class EventType(Enum):
    ARRIVAL = "arrival"
    SERVICE_COMPLETION = "service_completion"
    ABANDON = "abandon"
    CHARGING_COMPLETION = "charging_completion"

@dataclass
class Event:
    time: float
    event_type: EventType
    customer_id: Optional[int] = None
    server_id: Optional[int] = None
    
    def __lt__(self, other):
        return self.time < other.time

@dataclass
class Customer:
    id: int
    arrival_time: float
    service_time: float
    patience_time: float
    
    # Results to be filled during simulation
    queue_length_on_arrival: int = 0
    servers_available_on_arrival: int = 0
    servers_occupied_on_arrival: int = 0  # NEW: servers busy serving customers
    servers_charging_on_arrival: int = 0  # NEW: servers in charging state
    system_size_on_arrival: int = 0  # Total customers in system
    wait_time: float = 0.0
    service_start_time: float = 0.0
    departure_time: float = 0.0
    charging_time: float = 0.0  # Time spent in charging (if applicable)
    abandoned: bool = False

    def get_server_state_summary(self):
        """Get a summary of server states at arrival"""
        total = self.servers_available_on_arrival + self.servers_occupied_on_arrival + self.servers_charging_on_arrival
        return {
            'available': self.servers_available_on_arrival,
            'occupied': self.servers_occupied_on_arrival,
            'charging': self.servers_charging_on_arrival,
            'total': total
        }

class QueueSimulation:
    def __init__(self, num_customers: int, lambda_rate: float, mu_rate: float, 
                 theta_rate: float, num_servers: int, p_charge: float = 0.0, 
                 gamma_rate: float = 1.0, charging_distribution: str = 'exponential',random_seed: int = 2221):
        """
        Initialize queue simulation parameters
        
        Args:
            num_customers: Fixed number of customers to simulate
            lambda_rate: Arrival rate (λ)
            mu_rate: Service rate (μ)
            theta_rate: Abandonment rate (θ)
            num_servers: Total number of servers (c)
            p_charge: Probability of going to charge after service (p)
            gamma_rate: Charging completion rate (γ)
            random_seed: Random seed for reproducibility
        """
        self.num_customers = num_customers
        self.lambda_rate = lambda_rate
        self.mu_rate = mu_rate
        self.theta_rate = theta_rate
        self.num_servers = num_servers
        self.p_charge = p_charge
        self.gamma_rate = gamma_rate
        
        np.random.seed(random_seed)
        
        # Generate all random variables upfront
        self.customers = self._generate_customers()

        # Pre-generate charging decisions for server completions
        # We need more than num_customers because servers can complete multiple services
        max_service_completions = num_customers * 3  # Conservative estimate
        self.charging_decisions = np.random.binomial(1, self.p_charge, max_service_completions)
        #print(self.charging_decisions)
        if charging_distribution == 'exponential':
            self.charging_times = np.random.exponential(1/self.gamma_rate, max_service_completions)
        elif charging_distribution == 'constant':
            self.charging_times = np.ones(max_service_completions) * (1/self.gamma_rate)  # Fixed charging time for simplicity 
        elif charging_distribution == 'uniform':
            self.charging_times = np.random.uniform(0, 2/self.gamma_rate, max_service_completions) #need to fix this
        else:
            raise ValueError(f"Unknown charging distribution: {charging_distribution}")
        #print(self.charging_times)
        self.charging_decision_index = 0
        
    def _generate_customers(self) -> List[Customer]:
        """Generate all customers with their random attributes"""
        customers = []
        
        # Generate inter-arrival times (exponential)
        inter_arrivals = np.random.exponential(1/self.lambda_rate, self.num_customers)
        arrival_times = np.cumsum(inter_arrivals)
        
        # Generate service times (exponential)
        service_times = np.random.exponential(1/self.mu_rate, self.num_customers)
        
        # Generate patience times (exponential)
        patience_times = np.random.exponential(1/self.theta_rate, self.num_customers)
        
        
        for i in range(self.num_customers):
            customer = Customer(
                id=i,
                arrival_time=arrival_times[i],
                service_time=service_times[i],
                patience_time=patience_times[i]
            )
            customers.append(customer)
            
        return customers
    
    def simulate_erlang_a(self) -> List[Customer]:
        """Simulate Erlang-A queue (no charging)"""
        # Initialize state
        event_queue = []
        queue = []  # Waiting customers
        servers = [None] * self.num_servers  # None means idle, customer_id means busy
        current_time = 0.0
        
        # Schedule all arrivals
        for customer in self.customers:
            heapq.heappush(event_queue, Event(customer.arrival_time, EventType.ARRIVAL, customer.id))
        
        # Process events
        while event_queue:
            event = heapq.heappop(event_queue)
            current_time = event.time
            
            if event.event_type == EventType.ARRIVAL:
                self._handle_arrival_erlang_a(event, queue, servers, event_queue)
            elif event.event_type == EventType.SERVICE_COMPLETION:
                self._handle_service_completion_erlang_a(event, queue, servers, event_queue)
            elif event.event_type == EventType.ABANDON:
                self._handle_abandon(event, queue)
        
        return self.customers
    
    def simulate_erlang_s(self) -> List[Customer]:
        """Simulate Erlang-S* queue (with charging)"""
        # Initialize state
        event_queue = []
        queue = []  # Waiting customers
        active_servers = list(range(self.num_servers))  # All servers start active
        charging_servers = []  # Servers in charging state
        server_assignments = {}  # server_id -> customer_id
        current_time = 0.0
        
        #print(f"DEBUG: Starting simulation with {self.num_servers} servers")
        #print(f"DEBUG: Initial active_servers: {active_servers}")
        
        # Schedule all arrivals
        for customer in self.customers:
            heapq.heappush(event_queue, Event(customer.arrival_time, EventType.ARRIVAL, customer.id))
        
        # Process events
        while event_queue:
            event = heapq.heappop(event_queue)
            current_time = event.time
            
            if event.event_type == EventType.ARRIVAL:
                self._handle_arrival_erlang_s(event, queue, active_servers, server_assignments, event_queue, charging_servers)
            elif event.event_type == EventType.SERVICE_COMPLETION:
                self._handle_service_completion_erlang_s(event, queue, active_servers, charging_servers, 
                                                    server_assignments, event_queue)
            elif event.event_type == EventType.ABANDON:
                self._handle_abandon(event, queue)
            elif event.event_type == EventType.CHARGING_COMPLETION:
                # Pass all necessary parameters for serving waiting customers
                self._handle_charging_completion(event, active_servers, charging_servers, 
                                            queue, server_assignments, event_queue)
        
        #print(f"DEBUG: Final state - active: {len(active_servers)}, charging: {len(charging_servers)}, occupied: {len(server_assignments)}")
        return self.customers

    # Update the arrival handler to accept charging_servers
    def _handle_arrival_erlang_s(self, event, queue, active_servers, server_assignments, event_queue, charging_servers):
        customer = self.customers[event.customer_id]
        customer.queue_length_on_arrival = len(queue)
        
        # Track server states for Erlang-S*
        available_servers = len(active_servers)
        occupied_servers = len(server_assignments)
        charging_servers_count = len(charging_servers)
        
        customer.servers_available_on_arrival = available_servers
        customer.servers_occupied_on_arrival = occupied_servers
        customer.servers_charging_on_arrival = charging_servers_count
        
        # Verify server count consistency
        total_tracked = available_servers + occupied_servers + charging_servers_count
        if total_tracked != self.num_servers:
            print(f"DEBUG WARNING: Server count mismatch at time {event.time:.3f}")
            print(f"  Available: {available_servers}, Occupied: {occupied_servers}, Charging: {charging_servers_count}")
            print(f"  Total: {total_tracked}, Expected: {self.num_servers}")
        
        # Calculate total customers in system (queue + being served)
        customers_being_served = occupied_servers
        customer.system_size_on_arrival = len(queue) + customers_being_served
        
        if active_servers:
            # Start service immediately
            server_id = active_servers.pop(0)
            server_assignments[server_id] = customer.id
            customer.service_start_time = event.time
            customer.wait_time = 0.0
            
            # Schedule service completion
            completion_time = event.time + customer.service_time
            heapq.heappush(event_queue, Event(completion_time, EventType.SERVICE_COMPLETION, 
                                            customer.id, server_id))
        else:
            # Join queue and schedule abandonment
            queue.append(customer.id)
            abandon_time = event.time + customer.patience_time
            heapq.heappush(event_queue, Event(abandon_time, EventType.ABANDON, customer.id))
        
    def _handle_arrival_erlang_a(self, event, queue, servers, event_queue):
        customer = self.customers[event.customer_id]
        customer.queue_length_on_arrival = len(queue)
        
        # Track server states for Erlang-A
        available_servers = sum(1 for s in servers if s is None)
        occupied_servers = sum(1 for s in servers if s is not None)
        
        customer.servers_available_on_arrival = available_servers
        customer.servers_occupied_on_arrival = occupied_servers
        customer.servers_charging_on_arrival = 0  # No charging in Erlang-A
        
        # Calculate total customers in system (queue + being served)
        customers_being_served = occupied_servers
        customer.system_size_on_arrival = len(queue) + customers_being_served
        
        # Check if server is available
        available_server = None
        for i, server in enumerate(servers):
            if server is None:
                available_server = i
                break
        
        if available_server is not None:
            # Start service immediately
            servers[available_server] = customer.id
            customer.service_start_time = event.time
            customer.wait_time = 0.0
            
            # Schedule service completion
            completion_time = event.time + customer.service_time
            heapq.heappush(event_queue, Event(completion_time, EventType.SERVICE_COMPLETION, 
                                            customer.id, available_server))
        else:
            # Join queue and schedule abandonment
            queue.append(customer.id)
            abandon_time = event.time + customer.patience_time
            heapq.heappush(event_queue, Event(abandon_time, EventType.ABANDON, customer.id))

    def _handle_charging_completion(self, event, active_servers, charging_servers, queue=None, server_assignments=None, event_queue=None):
        """Handle server charging completion - move server back to active and serve waiting customers"""
        server_id = event.server_id
        if server_id in charging_servers:
            charging_servers.remove(server_id)
            
            # Check if there are waiting customers in queue
            if queue and len(queue) > 0 and server_assignments is not None and event_queue is not None:
                # Immediately serve next customer instead of going to active list
                next_customer_id = queue.pop(0)
                next_customer = self.customers[next_customer_id]
                
                # Assign server to customer
                server_assignments[server_id] = next_customer_id
                next_customer.service_start_time = event.time
                next_customer.wait_time = event.time - next_customer.arrival_time
                
                # Schedule service completion
                completion_time = event.time + next_customer.service_time
                heapq.heappush(event_queue, Event(completion_time, EventType.SERVICE_COMPLETION, 
                                                next_customer_id, server_id))
                
                #print(f"DEBUG: Server {server_id} finished charging at time {event.time:.3f} and immediately serves customer {next_customer_id}")
            else:
                # No waiting customers, add to active servers
                active_servers.append(server_id)
                #print(f"DEBUG: Server {server_id} finished charging at time {event.time:.3f} and becomes available")
            
            #print(f"DEBUG: Active servers now: {active_servers}")
            #print(f"DEBUG: Charging servers now: {charging_servers}")
        else:
            print(f"DEBUG: ERROR - Server {server_id} not found in charging_servers at time {event.time:.3f}")
            print(f"DEBUG: Current charging_servers: {charging_servers}")
            print(f"DEBUG: Current active_servers: {active_servers}")

    
    def _handle_service_completion_erlang_a(self, event, queue, servers, event_queue):
        customer = self.customers[event.customer_id]
        customer.departure_time = event.time
        servers[event.server_id] = None
        
        # Check if there's a waiting customer
        if queue:
            next_customer_id = queue.pop(0)
            next_customer = self.customers[next_customer_id]
            
            # Remove abandonment event for this customer (if it exists)
            # Note: In a more sophisticated implementation, we'd track and remove the event
            
            # Start service for next customer
            servers[event.server_id] = next_customer_id
            next_customer.service_start_time = event.time
            next_customer.wait_time = event.time - next_customer.arrival_time
            
            # Schedule service completion
            completion_time = event.time + next_customer.service_time
            heapq.heappush(event_queue, Event(completion_time, EventType.SERVICE_COMPLETION, 
                                            next_customer_id, event.server_id))
    
    def _handle_service_completion_erlang_s(self, event, queue, active_servers, charging_servers, server_assignments, event_queue):
        """Handle service completion in Erlang-S* system"""
        customer = self.customers[event.customer_id]
        customer.departure_time = event.time
        
        # Remove server from assignments
        if event.server_id in server_assignments:
            del server_assignments[event.server_id]
        else:
            print(f"DEBUG: WARNING - Server {event.server_id} not found in assignments")
        
        #print(f"DEBUG: Service completion for customer {customer.id} by server {event.server_id} at time {event.time:.3f}")
        #print(f"DEBUG: Queue length: {len(queue)}")
        
        # EVERY service completion should have a charging decision
        if self.charging_decision_index < len(self.charging_decisions):
            server_goes_to_charge = bool(self.charging_decisions[self.charging_decision_index])
            server_charging_time = self.charging_times[self.charging_decision_index]
            self.charging_decision_index += 1
            
            #print(f"DEBUG: Server {event.server_id} charging decision: {server_goes_to_charge}")
            
            if server_goes_to_charge:
                # Server goes to charging regardless of queue status
                charging_servers.append(event.server_id)
                #print(f"DEBUG: Server {event.server_id} starts charging for {server_charging_time:.3f} time units")
                
                # Schedule charging completion
                charging_completion_time = event.time + server_charging_time
                customer.charging_time = server_charging_time  # Record charging time for the customer
                heapq.heappush(event_queue, Event(charging_completion_time, EventType.CHARGING_COMPLETION, 
                                                None, event.server_id))
                #print(f"DEBUG: Charging completion scheduled for time {charging_completion_time:.3f}")
                
                # If there are waiting customers, they wait for this server to finish charging
                # OR other available servers can serve them
                
            else:
                # Server doesn't go charging, check if there are waiting customers
                if queue:
                    next_customer_id = queue.pop(0)
                    next_customer = self.customers[next_customer_id]
                    
                    # Use the same server that just finished
                    server_assignments[event.server_id] = next_customer_id
                    next_customer.service_start_time = event.time
                    next_customer.wait_time = event.time - next_customer.arrival_time
                    
                    #print(f"DEBUG: Server {event.server_id} immediately serves next customer {next_customer_id}")
                    
                    # Schedule service completion
                    completion_time = event.time + next_customer.service_time
                    heapq.heappush(event_queue, Event(completion_time, EventType.SERVICE_COMPLETION, 
                                                    next_customer_id, event.server_id))
                else:
                    # No waiting customers, server becomes available
                    active_servers.append(event.server_id)
                    #print(f"DEBUG: Server {event.server_id} becomes immediately available")
        else:
            # Fallback if we run out of pre-generated decisions
            if queue:
                next_customer_id = queue.pop(0)
                next_customer = self.customers[next_customer_id]
                server_assignments[event.server_id] = next_customer_id
                next_customer.service_start_time = event.time
                next_customer.wait_time = event.time - next_customer.arrival_time
                completion_time = event.time + next_customer.service_time
                heapq.heappush(event_queue, Event(completion_time, EventType.SERVICE_COMPLETION, 
                                                next_customer_id, event.server_id))
            else:
                active_servers.append(event.server_id)
                #print(f"DEBUG: Server {event.server_id} becomes available (no more charging decisions)")
        
        #print(f"DEBUG: After service completion - Active: {len(active_servers)}, Charging: {len(charging_servers)}, Occupied: {len(server_assignments)}")
        
    def _handle_abandon(self, event, queue):
        customer_id = event.customer_id
        if customer_id in queue:
            queue.remove(customer_id)
            customer = self.customers[customer_id]
            customer.abandoned = True
            customer.wait_time = event.time - customer.arrival_time
            customer.departure_time = event.time
    
    # def _handle_charging_completion(self, event, active_servers, charging_servers):
    #     server_id = event.server_id
    #     if server_id in charging_servers:
    #         charging_servers.remove(server_id)
    #         active_servers.append(server_id)


In [3]:
def analyze_and_visualize(customers_a: List[Customer], customers_s: List[Customer], 
                         params: Dict, save_plots: bool = False):
    """Analyze simulation results and create visualizations"""
    
    # Create DataFrames for analysis
    df_a = pd.DataFrame([{
        'customer_id': c.id,
        'arrival_time': c.arrival_time,
        'queue_length_on_arrival': c.queue_length_on_arrival,
        'system_size_on_arrival': c.system_size_on_arrival,
        'servers_available_on_arrival': c.servers_available_on_arrival,
        'wait_time': c.wait_time,
        'service_time': c.service_time,
        'departure_time': c.departure_time,
        'abandoned': c.abandoned,
        'system': 'Erlang-A'
    } for c in customers_a])
    
    df_s = pd.DataFrame([{
        'customer_id': c.id,
        'arrival_time': c.arrival_time,
        'queue_length_on_arrival': c.queue_length_on_arrival,
        'system_size_on_arrival': c.system_size_on_arrival,
        'servers_available_on_arrival': c.servers_available_on_arrival,
        'wait_time': c.wait_time,
        'service_time': c.service_time,
        'departure_time': c.departure_time,
        'abandoned': c.abandoned,
        'system': 'Erlang-S*'
    } for c in customers_s])
    
    df_combined = pd.concat([df_a, df_s], ignore_index=True)
    
    # Print summary statistics
    print("=== SIMULATION RESULTS ===")
    print(f"Parameters: λ={params['lambda']}, μ={params['mu']}, θ={params['theta']}, c={params['c']}")
    print(f"Erlang-S* specific: p={params['p']}, γ={params['gamma']}")
    print("\nERLANG-A QUEUE:")
    print(f"  Average wait time: {df_a['wait_time'].mean():.3f}")
    print(f"  Average queue length on arrival: {df_a['queue_length_on_arrival'].mean():.3f}")
    print(f"  Abandonment rate: {df_a['abandoned'].mean():.3f}")
    print(f"  Average servers available on arrival: {df_a['servers_available_on_arrival'].mean():.3f}")
    
    print("\nERLANG-S* QUEUE:")
    print(f"  Average wait time: {df_s['wait_time'].mean():.3f}")
    print(f"  Average queue length on arrival: {df_s['queue_length_on_arrival'].mean():.3f}")
    print(f"  Abandonment rate: {df_s['abandoned'].mean():.3f}")
    print(f"  Average servers available on arrival: {df_s['servers_available_on_arrival'].mean():.3f}")
    
    # Create comprehensive visualizations
    fig = plt.figure(figsize=(20, 16))
    
    # 1. Wait Time Distributions
    plt.subplot(3, 4, 1)
    non_abandoned_a = df_a[~df_a['abandoned']]['wait_time']
    non_abandoned_s = df_s[~df_s['abandoned']]['wait_time']
    
    plt.hist(non_abandoned_a, alpha=0.7, bins=30, label='Erlang-A', density=True)
    plt.hist(non_abandoned_s, alpha=0.7, bins=30, label='Erlang-S*', density=True)
    plt.xlabel('Wait Time')
    plt.ylabel('Density')
    plt.title('Wait Time Distribution\n(Non-abandoned customers)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 2. Queue Length on Arrival
    plt.subplot(3, 4, 2)
    queue_lengths_a = df_a['queue_length_on_arrival'].value_counts().sort_index()
    queue_lengths_s = df_s['queue_length_on_arrival'].value_counts().sort_index()
    
    x_pos = np.arange(max(queue_lengths_a.index.max(), queue_lengths_s.index.max()) + 1)
    plt.bar(x_pos - 0.2, [queue_lengths_a.get(i, 0) for i in x_pos], 
            width=0.4, alpha=0.7, label='Erlang-A')
    plt.bar(x_pos + 0.2, [queue_lengths_s.get(i, 0) for i in x_pos], 
            width=0.4, alpha=0.7, label='Erlang-S*')
    plt.xlabel('Queue Length on Arrival')
    plt.ylabel('Frequency')
    plt.title('Queue Length Distribution\n(On Arrival)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 3. Servers Available on Arrival
    plt.subplot(3, 4, 3)
    servers_a = df_a['servers_available_on_arrival'].value_counts().sort_index()
    servers_s = df_s['servers_available_on_arrival'].value_counts().sort_index()
    
    x_pos = np.arange(params['c'] + 1)
    plt.bar(x_pos - 0.2, [servers_a.get(i, 0) for i in x_pos], 
            width=0.4, alpha=0.7, label='Erlang-A')
    plt.bar(x_pos + 0.2, [servers_s.get(i, 0) for i in x_pos], 
            width=0.4, alpha=0.7, label='Erlang-S*')
    plt.xlabel('Servers Available on Arrival')
    plt.ylabel('Frequency')
    plt.title('Available Servers Distribution\n(On Arrival)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 4. Abandonment Comparison
    plt.subplot(3, 4, 4)
    abandon_rates = [df_a['abandoned'].mean(), df_s['abandoned'].mean()]
    systems = ['Erlang-A', 'Erlang-S*']
    colors = ['#1f77b4', '#ff7f0e']
    
    bars = plt.bar(systems, abandon_rates, color=colors, alpha=0.7)
    plt.ylabel('Abandonment Rate')
    plt.title('Abandonment Rate Comparison')
    plt.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, rate in zip(bars, abandon_rates):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{rate:.3f}', ha='center', va='bottom')
    
    # 5. Wait Time Over Time
    plt.subplot(3, 4, 5)
    window_size = max(50, len(customers_a) // 20)
    df_a_sorted = df_a.sort_values('arrival_time')
    df_s_sorted = df_s.sort_values('arrival_time')
    
    wait_time_ma_a = df_a_sorted['wait_time'].rolling(window=window_size, center=True).mean()
    wait_time_ma_s = df_s_sorted['wait_time'].rolling(window=window_size, center=True).mean()
    
    plt.plot(df_a_sorted['arrival_time'], wait_time_ma_a, label='Erlang-A', alpha=0.8)
    plt.plot(df_s_sorted['arrival_time'], wait_time_ma_s, label='Erlang-S*', alpha=0.8)
    plt.xlabel('Arrival Time')
    plt.ylabel('Wait Time (Moving Average)')
    plt.title(f'Wait Time Evolution\n(Moving Average, window={window_size})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 6. Service Time vs Wait Time Scatter
    plt.subplot(3, 4, 6)
    non_abandoned_a = df_a[~df_a['abandoned']]
    non_abandoned_s = df_s[~df_s['abandoned']]
    
    plt.scatter(non_abandoned_a['service_time'], non_abandoned_a['wait_time'], 
               alpha=0.5, label='Erlang-A', s=20)
    plt.scatter(non_abandoned_s['service_time'], non_abandoned_s['wait_time'], 
               alpha=0.5, label='Erlang-S*', s=20)
    plt.xlabel('Service Time')
    plt.ylabel('Wait Time')
    plt.title('Service Time vs Wait Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 7. Queue Length Evolution
    plt.subplot(3, 4, 7)
    plt.plot(df_a_sorted['arrival_time'], 
             df_a_sorted['queue_length_on_arrival'].rolling(window=window_size, center=True).mean(),
             label='Erlang-A', alpha=0.8)
    plt.plot(df_s_sorted['arrival_time'], 
             df_s_sorted['queue_length_on_arrival'].rolling(window=window_size, center=True).mean(),
             label='Erlang-S*', alpha=0.8)
    plt.xlabel('Arrival Time')
    plt.ylabel('Queue Length (Moving Average)')
    plt.title(f'Queue Length Evolution\n(Moving Average, window={window_size})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 8. Wait Time Box Plot
    plt.subplot(3, 4, 8)
    wait_data = [non_abandoned_a['wait_time'], non_abandoned_s['wait_time']]
    plt.boxplot(wait_data, labels=['Erlang-A', 'Erlang-S*'])
    plt.ylabel('Wait Time')
    plt.title('Wait Time Distribution\n(Box Plot)')
    plt.grid(True, alpha=0.3)
    
    # 9. Probability of Delay (P(Wait > 0))
    plt.subplot(3, 4, 9)
    delay_prob_a = (df_a['wait_time'] > 0).sum() / len(customers_a)
    delay_prob_s = (df_s['wait_time'] > 0).sum() / len(customers_s)
    
    bars = plt.bar(['Erlang-A', 'Erlang-S*'], [delay_prob_a, delay_prob_s], 
                   color=colors, alpha=0.7)
    plt.ylabel('Probability of Delay')
    plt.title('Probability of Delay\nP(Wait Time > 0)')
    plt.grid(True, alpha=0.3)
    
    for bar, prob in zip(bars, [delay_prob_a, delay_prob_s]):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{prob:.3f}', ha='center', va='bottom')
  
    
    # 10. System Time Distribution (Wait + Service)
    plt.subplot(3, 4, 10)
    system_time_a = non_abandoned_a['wait_time'] + non_abandoned_a['service_time']
    system_time_s = non_abandoned_s['wait_time'] + non_abandoned_s['service_time']
    
    plt.hist(system_time_a, alpha=0.7, bins=30, label='Erlang-A', density=True)
    plt.hist(system_time_s, alpha=0.7, bins=30, label='Erlang-S*', density=True)
    plt.xlabel('System Time (Wait + Service)')
    plt.ylabel('Density')
    plt.title('System Time Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 11. Utilization Over Time (approximation)
    plt.subplot(3, 4, 11)
    busy_servers_a = params['c'] - df_a_sorted['servers_available_on_arrival']
    busy_servers_s = params['c'] - df_s_sorted['servers_available_on_arrival']
    
    utilization_a = busy_servers_a.rolling(window=window_size, center=True).mean() / params['c']
    utilization_s = busy_servers_s.rolling(window=window_size, center=True).mean() / params['c']
    
    plt.plot(df_a_sorted['arrival_time'], utilization_a, label='Erlang-A', alpha=0.8)
    plt.plot(df_s_sorted['arrival_time'], utilization_s, label='Erlang-S*', alpha=0.8)
    plt.xlabel('Time')
    plt.ylabel('Utilization (approx)')
    plt.title('Server Utilization Over Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 12. Performance Metrics Summary
    plt.subplot(3, 4, 12)
    metrics = ['Avg Wait Time', 'Abandonment Rate', 'Delay Probability', 'Avg Queue Length']
    erlang_a_metrics = [
        df_a['wait_time'].mean(),
        df_a['abandoned'].mean(),
        delay_prob_a,
        df_a['queue_length_on_arrival'].mean()
    ]
    erlang_s_metrics = [
        df_s['wait_time'].mean(),
        df_s['abandoned'].mean(),
        delay_prob_s,
        df_s['queue_length_on_arrival'].mean()
    ]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.bar(x - width/2, erlang_a_metrics, width, label='Erlang-A', alpha=0.7)
    plt.bar(x + width/2, erlang_s_metrics, width, label='Erlang-S*', alpha=0.7)
    
    plt.xlabel('Metrics')
    plt.ylabel('Values')
    plt.title('Performance Metrics Comparison')
    plt.xticks(x, metrics, rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig('queue_simulation_analysis.png', dpi=300, bbox_inches='tight')
    
    plt.show()
    
    return df_combined

In [4]:
def delay_probability(customers_a: List[Customer], customers_s: List[Customer], params: Dict):
    """
    Calculate the probability of delay (waiting time > 0) for all customers.
    
    Args:
        customers_a: List of Customer objects from Erlang-A
        customers_s: List of Customer objects from Erlang-S*
        params: Dictionary of parameters
    
    Returns:
        Tuple of delay probabilities (Erlang-A, Erlang-S*)
    """
    # Create DataFrames for analysis
    df_a = pd.DataFrame([{
        'customer_id': c.id,
        'arrival_time': c.arrival_time,
        'queue_length_on_arrival': c.queue_length_on_arrival,
        'system_size_on_arrival': c.system_size_on_arrival,
        'servers_available_on_arrival': c.servers_available_on_arrival,
        'wait_time': c.wait_time,
        'service_time': c.service_time,
        'departure_time': c.departure_time,
        'abandoned': c.abandoned,
        'system': 'Erlang-A'
    } for c in customers_a])
    
    df_s = pd.DataFrame([{
        'customer_id': c.id,
        'arrival_time': c.arrival_time,
        'queue_length_on_arrival': c.queue_length_on_arrival,
        'system_size_on_arrival': c.system_size_on_arrival,
        'servers_available_on_arrival': c.servers_available_on_arrival,
        'wait_time': c.wait_time,
        'service_time': c.service_time,
        'departure_time': c.departure_time,
        'abandoned': c.abandoned,
        'system': 'Erlang-S*'
    } for c in customers_s])
    
    # colors = ['#1f77b4', '#ff7f0e']
    
    # plt.figure(figsize=(5, 4))
    
    # # FIXED: Calculate delay probability correctly
    delay_prob_a = (df_a['wait_time'] > 0).sum() / len(customers_a)
    delay_prob_s = (df_s['wait_time'] > 0).sum() / len(customers_s)

    abandoned_a = df_a['abandoned'].mean()
    abandoned_s = df_s['abandoned'].mean()
    
    # bars = plt.bar(['Erlang-A', 'Erlang-S*'], [delay_prob_a, delay_prob_s], 
    #                color=colors, alpha=0.7)
    # plt.ylabel('Probability of Delay')
    # plt.title('Probability of Delay\nP(Wait Time > 0)')
    # plt.grid(True, alpha=0.3)
    
    # # Add value labels on bars
    # for bar, prob in zip(bars, [delay_prob_a, delay_prob_s]):
    #     plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
    #             f'{prob:.3f}', ha='center', va='bottom')
    
    # plt.show()
    
    return delay_prob_a, delay_prob_s, abandoned_a, abandoned_s

In [5]:
if __name__ == "__main__":
    # Simulation parameters
    params = {
        'lambda': 100.0,    # Arrival rate
        'mu': 1,        # Service rate  
        'theta': 1,     # Abandonment rate
        'c': 100,           # Number of servers
        'p': 0.5,         # Probability of going to charge (Erlang-S* only)
        'gamma': 1      # Charging completion rate (Erlang-S* only)
    }
    
    num_customers = int(1000e3)
    
    print("Starting queue simulations...")
    print(f"Simulating {num_customers} customers with parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    
    # Run Erlang-A simulation
    print("\nRunning Erlang-A simulation...")
    sim_a = QueueSimulation(
        num_customers=num_customers,
        lambda_rate=params['lambda'],
        mu_rate=params['mu'],
        theta_rate=params['theta'],
        num_servers=params['c'],
        p_charge=0.0,  # No charging in Erlang-A
        gamma_rate=params['gamma'],
        charging_distribution= 'exponential'
    )
    customers_a = sim_a.simulate_erlang_a()
    
    # Run Erlang-S* simulation
    print("Running Erlang-S* simulation...")
    sim_s = QueueSimulation(
        num_customers=num_customers,
        lambda_rate=params['lambda'],
        mu_rate=params['mu'],
        theta_rate=params['theta'],
        num_servers=params['c'],
        p_charge=params['p'],
        gamma_rate=params['gamma'],
        charging_distribution= 'exponential'
    )
    customers_s = sim_s.simulate_erlang_s()
    
    # Analyze and visualize results
    print("Analyzing results and creating visualizations...")
    #df_results = analyze_and_visualize(customers_a, customers_s, params, save_plots=True)
    df_delay = delay_probability(customers_a, customers_s, params)
    print(df_delay)
    print("\nSimulation completed! Check the generated plots for detailed analysis.")

Starting queue simulations...
Simulating 1000000 customers with parameters:
  lambda: 100.0
  mu: 1
  theta: 1
  c: 100
  p: 0.5
  gamma: 1

Running Erlang-A simulation...
Running Erlang-S* simulation...
Analyzing results and creating visualizations...
(0.511659, 0.99969, 0.039526, 0.332242)

Simulation completed! Check the generated plots for detailed analysis.


In [8]:
for customer in customers_a:
    print(customer)
    if customer.id > 100:
        break

Customer(id=0, arrival_time=0.01017014465346594, service_time=0.37304368307483005, patience_time=1.9671344309108598, queue_length_on_arrival=0, servers_available_on_arrival=100, servers_occupied_on_arrival=0, servers_charging_on_arrival=0, system_size_on_arrival=0, wait_time=0.0, service_start_time=0.01017014465346594, departure_time=0.383213827728296, charging_time=0.0, abandoned=False)
Customer(id=1, arrival_time=0.012818410372705362, service_time=0.9914815262332335, patience_time=1.441875206355855, queue_length_on_arrival=0, servers_available_on_arrival=99, servers_occupied_on_arrival=1, servers_charging_on_arrival=0, system_size_on_arrival=1, wait_time=0.0, service_start_time=0.012818410372705362, departure_time=1.0042999366059389, charging_time=0.0, abandoned=False)
Customer(id=2, arrival_time=0.024245841486113015, service_time=0.20143648143513243, patience_time=0.030999637623488986, queue_length_on_arrival=0, servers_available_on_arrival=98, servers_occupied_on_arrival=2, servers